In [1]:
import scanpy as sc, anndata as ad, numpy as np, pandas as pd
from scipy import sparse
from anndata import AnnData
import warnings
import socket
from matplotlib import pylab
import sys
import yaml
import os
import matplotlib.pyplot as plt
import scvelo as scv
from pybiomart import Server
from tqdm.notebook import tqdm
import itertools
import plotly.express as px
import math

import random

warnings.filterwarnings('ignore')


# In[2]:


get_ipython().run_line_magic('matplotlib', 'inline')


# In[3]:


sc.settings.verbosity = 3         # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=50, facecolor='white')
pylab.rcParams['figure.figsize'] = (4, 4)

scanpy==1.8.1 anndata==0.7.6 umap==0.4.6 numpy==1.20.2 scipy==1.6.3 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.13.1 python-igraph==0.9.8 louvain==0.7.1 pynndescent==0.5.5


### Configure paths

In [2]:

outdir="./outdir"



# Cycling Curated

In [3]:
Finaleaf="/Cycling"
adata = sc.read_h5ad(outdir+Finaleaf+"/Badhuri_cycling_prebulk.Curated.h5ad")
sc.pp.calculate_qc_metrics(adata,  percent_top=None, log1p=False, inplace=True)

adata

adata.obs["individual_region"] = adata.obs["individual"].astype(str)+"_"+ adata.obs["cortical_area"].astype(str)
adata.obs["individual_region"] = adata.obs["individual_region"].astype("category")
adata.obs["individual_region"]

adata.obs.groupby(["cortical_area","individual"]).size()

cortical_area  individual
motor          GW14            83
               GW16          1377
               GW17           254
               GW18            14
               GW18_2         867
                             ... 
v1             GW20_31        556
               GW20_34        373
               GW22           253
               GW22T            0
               GW25             0
Length: 72, dtype: int64

## Groupingfactor = cortical_area

In [4]:
groupingCovariate = "cortical_area"
PseudooReplicates_per_group = 20
print("Pbulking with target of "+str(PseudooReplicates_per_group)+" PRs will result in following cells per PR")
adata.obs.groupby(groupingCovariate).size() / PseudooReplicates_per_group


Pbulking with target of 20 PRs will result in following cells per PR


cortical_area
motor            193.35
parietal         235.85
pfc              170.15
somatosensory    188.65
temporal         184.25
v1               251.40
dtype: float64

In [5]:
total = pd.DataFrame(index = adata.var_names)
total_metadata = pd.DataFrame()
for group in adata.obs[groupingCovariate].cat.categories:

    groupAdata = adata[adata.obs[groupingCovariate] == group].copy()
    
#    if groupAdata.shape[0] < 500:
#        continue
    

    group_cells = adata[adata.obs[groupingCovariate] == group].obs_names.tolist()
    random.shuffle(group_cells)

    metaCellslist=[group_cells[i::PseudooReplicates_per_group] for i in range(PseudooReplicates_per_group)]

    for partition in enumerate(metaCellslist):

        total['{}_{}'.format(group, partition[0])] = adata[partition[1]].X.sum(axis = 0).A1

        total_metadata.loc['{}_{}'.format(group, partition[0]),"group"] = group
        total_metadata.loc['{}_{}'.format(group, partition[0]),"pseudoreplicate"] = partition[0]
        total_metadata.loc['{}_{}'.format(group, partition[0]),"number_of_cell"] = int(len(partition[1]))
    
    for obsMD in [obsMD for obsMD in groupAdata.obs.columns.tolist() if len(groupAdata.obs[obsMD].unique()) == 1 and obsMD != groupingCovariate]:
        total_metadata.loc[["_".join(l) for l in list(itertools.product([group],[str(r)  for r in list(range(PseudooReplicates_per_group))]))], obsMD ] = adata.obs.loc[adata.obs[groupingCovariate] == group,obsMD][0]
        
        
total_metadata = total_metadata.dropna(axis = 1)

In [6]:
adata_bulk = sc.AnnData(total.transpose()).copy()
adata_bulk.var = adata.var.copy()
adata_bulk.obs = pd.concat([adata_bulk.obs, total_metadata], axis = 1)


with open("./colorMap.yaml", 'r') as f:
    colorMap = yaml.load(f, Loader=yaml.FullLoader)["uns_colors"]
colorMap

adata_bulk.obs["group"] = adata_bulk.obs["group"].astype("category")
adata_bulk.uns["group_colors"] = [colorMap[i]["color"] for i in adata_bulk.obs["group"].cat.categories]





del adata_bulk.obs["is_primary_data"]

In [7]:
adata_bulk.write_h5ad(outdir+Finaleaf+"/Badhuri_cycling_pBulk.Curated."+str(PseudooReplicates_per_group)+"PRs.by"+str(groupingCovariate)+".h5ad")
total.to_csv(outdir+Finaleaf+"/Badhuri_cycling_pBulk.Curated."+str(PseudooReplicates_per_group)+"PRs.by"+str(groupingCovariate)+".tsv", sep="\t")

... storing 'brain_region' as categorical
... storing 'tissue_ontology_term_id' as categorical
... storing 'assay_ontology_term_id' as categorical
... storing 'disease_ontology_term_id' as categorical
... storing 'cell_type_ontology_term_id' as categorical
... storing 'ethnicity_ontology_term_id' as categorical
... storing 'organism_ontology_term_id' as categorical
... storing 'sex_ontology_term_id' as categorical
... storing 'cell_type' as categorical
... storing 'assay' as categorical
... storing 'disease' as categorical
... storing 'organism' as categorical
... storing 'sex' as categorical
... storing 'tissue' as categorical
... storing 'ethnicity' as categorical
... storing 'cluster_label_aggr' as categorical
... storing 'cluster_label_area' as categorical
... storing 'cluster_label_area_layers' as categorical


# Neurons prebulk

In [13]:
FinaLeaf="/Neurons"
adata = sc.read_h5ad(outdir+FinaLeaf+"/Badhuri_Neurons_prebulk.Curated.h5ad")
sc.pp.calculate_qc_metrics(adata,  percent_top=None, log1p=False, inplace=True)

adata.obs["individual_region"] = adata.obs["individual"].astype(str)+"_"+ adata.obs["cortical_area"].astype(str)
adata.obs["individual_region"] = adata.obs["individual_region"].astype("category")
adata.obs["individual_region"]

adata.obs.groupby(["cortical_area","individual"]).size()

cortical_area  individual
motor          GW14            92
               GW17           405
               GW18          5499
               GW19          1343
               GW20             8
               GW20_31          0
               GW20_34          2
               GW22           637
               GW22T          621
               GW25           317
parietal       GW14             0
               GW17             2
               GW18             2
               GW19          1593
               GW20             0
               GW20_31       1592
               GW20_34       3096
               GW22           524
               GW22T          282
               GW25          6766
pfc            GW14             0
               GW17           348
               GW18          3522
               GW19          2499
               GW20          2187
               GW20_31       3297
               GW20_34       2014
               GW22          1140
               GW22T  

## Groupingfactor = cortical_area

In [14]:
groupingCovariate = "cortical_area"
PseudooReplicates_per_group = 20
print("Pbulking with target of "+str(PseudooReplicates_per_group)+" PRs will result in following cells per PR")
adata.obs.groupby(groupingCovariate).size() / PseudooReplicates_per_group


Pbulking with target of 20 PRs will result in following cells per PR


cortical_area
motor             446.20
parietal          692.85
pfc              1108.75
somatosensory     192.05
temporal          155.55
v1                571.05
dtype: float64

In [15]:
total = pd.DataFrame(index = adata.var_names)
total_metadata = pd.DataFrame()
for group in adata.obs[groupingCovariate].cat.categories:

    groupAdata = adata[adata.obs[groupingCovariate] == group].copy()
    
#    if groupAdata.shape[0] < 500:
#        continue
    
    

    group_cells = adata[adata.obs[groupingCovariate] == group].obs_names.tolist()
    random.shuffle(group_cells)

    metaCellslist=[group_cells[i::PseudooReplicates_per_group] for i in range(PseudooReplicates_per_group)]

    for partition in enumerate(metaCellslist):

        total['{}_{}'.format(group, partition[0])] = adata[partition[1]].X.sum(axis = 0).A1

        total_metadata.loc['{}_{}'.format(group, partition[0]),"group"] = group
        total_metadata.loc['{}_{}'.format(group, partition[0]),"pseudoreplicate"] = partition[0]
        total_metadata.loc['{}_{}'.format(group, partition[0]),"number_of_cell"] = int(len(partition[1]))
    
    for obsMD in [obsMD for obsMD in groupAdata.obs.columns.tolist() if len(groupAdata.obs[obsMD].unique()) == 1 and obsMD != groupingCovariate]:
        total_metadata.loc[["_".join(l) for l in list(itertools.product([group],[str(r)  for r in list(range(PseudooReplicates_per_group))]))], obsMD ] = adata.obs.loc[adata.obs[groupingCovariate] == group,obsMD][0]
        
        
total_metadata = total_metadata.dropna(axis = 1)

In [16]:
adata_bulk = sc.AnnData(total.transpose()).copy()
adata_bulk.var = adata.var.copy()
adata_bulk.obs = pd.concat([adata_bulk.obs, total_metadata], axis = 1)


adata_bulk.obs["group"] = adata_bulk.obs["group"].astype("category")
adata_bulk.uns["group_colors"] = [colorMap[i]["color"] for i in adata_bulk.obs["group"].cat.categories]

del adata_bulk.obs["is_primary_data"]

In [17]:
adata_bulk.write_h5ad(outdir+FinaLeaf+"/Badhuri_neurons_pBulk.Curated."+str(PseudooReplicates_per_group)+"PRs.by"+str(groupingCovariate)+".h5ad")
total.to_csv(outdir+FinaLeaf+"/Badhuri_neurons_pBulk.Curated."+str(PseudooReplicates_per_group)+"PRs.by"+str(groupingCovariate)+".tsv", sep="\t")

... storing 'brain_region' as categorical
... storing 'tissue_ontology_term_id' as categorical
... storing 'assay_ontology_term_id' as categorical
... storing 'disease_ontology_term_id' as categorical
... storing 'cell_type_ontology_term_id' as categorical
... storing 'ethnicity_ontology_term_id' as categorical
... storing 'organism_ontology_term_id' as categorical
... storing 'sex_ontology_term_id' as categorical
... storing 'cell_type' as categorical
... storing 'assay' as categorical
... storing 'disease' as categorical
... storing 'organism' as categorical
... storing 'sex' as categorical
... storing 'tissue' as categorical
... storing 'ethnicity' as categorical
... storing 'cluster_label_aggr' as categorical
... storing 'batch' as categorical
... storing 'cluster_label_area' as categorical
... storing 'GeschIngestedAnno' as categorical
... storing 'cluster_label_area_layers' as categorical


# Progenitors prebulk

In [18]:
FinaLeaf="/Progenitors"
adata = sc.read_h5ad(outdir+FinaLeaf+"/Badhuri_progenitors_prebulk.Curated.h5ad")
sc.pp.calculate_qc_metrics(adata,  percent_top=None, log1p=False, inplace=True)

adata.obs["individual_region"] = adata.obs["individual"].astype(str)+"_"+ adata.obs["cortical_area"].astype(str)
adata.obs["individual_region"] = adata.obs["individual_region"].astype("category")
adata.obs["individual_region"]

adata.obs.groupby(["cortical_area","individual"]).size()

cortical_area  individual
motor          GW14            2
               GW16          287
               GW17           10
               GW18            0
               GW18_2        392
                            ... 
v1             GW20_31       208
               GW20_34        76
               GW22           45
               GW22T           0
               GW25            0
Length: 72, dtype: int64

## Groupingfactor = cortical_area

In [19]:
groupingCovariate = "cortical_area"
PseudooReplicates_per_group = 20
print("Pbulking with target of "+str(PseudooReplicates_per_group)+" PRs will result in following cells per PR")
adata.obs.groupby(groupingCovariate).size() / PseudooReplicates_per_group


Pbulking with target of 20 PRs will result in following cells per PR


cortical_area
motor             55.20
parietal         110.00
pfc               48.30
somatosensory     55.25
temporal          90.15
v1                96.40
dtype: float64

In [20]:
total = pd.DataFrame(index = adata.var_names)
total_metadata = pd.DataFrame()
for group in adata.obs[groupingCovariate].cat.categories:

    groupAdata = adata[adata.obs[groupingCovariate] == group].copy()
    
#    if groupAdata.shape[0] < 500:
#        continue
    

    group_cells = adata[adata.obs[groupingCovariate] == group].obs_names.tolist()
    random.shuffle(group_cells)

    metaCellslist=[group_cells[i::PseudooReplicates_per_group] for i in range(PseudooReplicates_per_group)]

    for partition in enumerate(metaCellslist):

        total['{}_{}'.format(group, partition[0])] = adata[partition[1]].X.sum(axis = 0).A1

        total_metadata.loc['{}_{}'.format(group, partition[0]),"group"] = group
        total_metadata.loc['{}_{}'.format(group, partition[0]),"pseudoreplicate"] = partition[0]
        total_metadata.loc['{}_{}'.format(group, partition[0]),"number_of_cell"] = int(len(partition[1]))
    
    for obsMD in [obsMD for obsMD in groupAdata.obs.columns.tolist() if len(groupAdata.obs[obsMD].unique()) == 1 and obsMD != groupingCovariate]:
        total_metadata.loc[["_".join(l) for l in list(itertools.product([group],[str(r)  for r in list(range(PseudooReplicates_per_group))]))], obsMD ] = adata.obs.loc[adata.obs[groupingCovariate] == group,obsMD][0]
        
        
total_metadata = total_metadata.dropna(axis = 1)

In [21]:
adata_bulk = sc.AnnData(total.transpose()).copy()
adata_bulk.var = adata.var.copy()
adata_bulk.obs = pd.concat([adata_bulk.obs, total_metadata], axis = 1)

adata_bulk.obs["group"] = adata_bulk.obs["group"].astype("category")
adata_bulk.uns["group_colors"] = [colorMap[i]["color"] for i in adata_bulk.obs["group"].cat.categories]

del adata_bulk.obs["is_primary_data"]

In [22]:
adata_bulk.write_h5ad(outdir+FinaLeaf+"/Badhuri_progenitors_pBulk.Curated."+str(PseudooReplicates_per_group)+"PRs.by"+str(groupingCovariate)+".h5ad")
total.to_csv(outdir+FinaLeaf+"/Badhuri_progenitors_pBulk.Curated."+str(PseudooReplicates_per_group)+"PRs.by"+str(groupingCovariate)+".tsv", sep="\t")

... storing 'brain_region' as categorical
... storing 'tissue_ontology_term_id' as categorical
... storing 'assay_ontology_term_id' as categorical
... storing 'disease_ontology_term_id' as categorical
... storing 'cell_type_ontology_term_id' as categorical
... storing 'ethnicity_ontology_term_id' as categorical
... storing 'organism_ontology_term_id' as categorical
... storing 'sex_ontology_term_id' as categorical
... storing 'cell_type' as categorical
... storing 'assay' as categorical
... storing 'disease' as categorical
... storing 'organism' as categorical
... storing 'sex' as categorical
... storing 'tissue' as categorical
... storing 'ethnicity' as categorical
... storing 'cluster_label_aggr' as categorical
... storing 'cluster_label_area' as categorical
... storing 'GeschIngestedAnno' as categorical
... storing 'cluster_label_area_layers' as categorical
